<a href="https://colab.research.google.com/github/serrabaysal/auto-ml-course-notes/blob/main/13aral%C4%B1k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers NLP'den Vision'a
RNN:Veriyi soldan sağa sıralı bir şekilde okur.
1. RNN neden yetersiz kaldı: Kısa cümlelerle sorunsuz çalışırken uzun cümlelerin başını unutuyordu.Sadece yakın geçmişi hatırlayabiliyordu.
Sonra LSTM ve GRU yaklaşımları bulundu.
2. 2017'de RNN ve benzeri kavramların tamamı geçerliliğini yitiriyor. "Attention Is All You Need" mekanizması ortaya çıkıyor.
3. Transformers Mimarisi
    *   Encoder(Kodlayıcı)Anlayan:Cümlelerin tamamına aynı anda bakar.NLP(doğal dil işleme).
    *   Decoder(Çözücü)Üreten: GPT-3,GPT-4 gibi hem anlayıp hem de üretebiliyorlar.
4. Self-Attention:Veritabanı Antolojisi
5. Bağlamı Anlamak
# NLP
1. BERT:Anlam Uzmanı
2. Duygu analizi,anlam çıkarma konusunda uzmandır. MLM:Masked Language Modeling
3. GPT:Üretim Uzmanı
4. Tokenizasyon:Metinden Sayıya
# Computer Vision


In [ ]:
# ==============================================================================
# WORKSHOP BÖLÜM 1: NLP - Zero-Shot Sınıflandırma
# Hedef Kitle: Bilgisayar Mühendisliği Öğrencileri
# Amaç: Etiketli veri seti olmadan (training yapmadan) metin sınıflandırmak.
# ==============================================================================

# ÖNCE GEREKLİ KÜTÜPHANELERİ YÜKLEYELİM
# Google Colab'de en üstteki hücreye şunu yazıp çalıştırın:
# !pip install transformers gradio sentencepiece

import gradio as gr
from transformers import pipeline

print("Model yükleniyor... (Bu işlem modelin boyutuna göre 1-2 dakika sürebilir)")

# ------------------------------------------------------------------------------
# ADIM 1: MODELİ VE TOKENIZER'I HAZIRLAMAK
# ------------------------------------------------------------------------------
# Burada Hugging Face 'pipeline' yapısını kullanıyoruz.
# Pipeline, karmaşık işlemleri (Tokenization -> Model Inference -> Post-Processing)
# tek bir satıra indirger.
#
# Kullandığımız Model: "joeddav/xlm-roberta-large-xnli"
# Neden bu model?
# 1. "XLM-RoBERTa": Çok dilli (Multilingual) bir modeldir, Türkçeyi iyi anlar.
# 2. "XNLI": Cross-lingual Natural Language Inference veri setinde eğitilmiştir.
#    Yani iki cümlenin birbirini doğrulayıp doğrulamadığını anlar.
# ------------------------------------------------------------------------------

classifier = pipeline(
    task="zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli"
)

print("Model başarıyla yüklendi! Arayüz başlatılıyor...")

# ------------------------------------------------------------------------------
# ADIM 2: SINIFLANDIRMA FONKSİYONU
# ------------------------------------------------------------------------------
def siniflandir(text, labels_text):
    """
    Bu fonksiyon arayüzden gelen metni ve etiketleri alır, modele verir.

    Args:
        text (str): Sınıflandırılacak cümle.
        labels_text (str): Virgülle ayrılmış etiketler (örn: "Spor, Siyaset").

    Returns:
        dict: Gradio için etiket ve olasılık değerlerini içeren sözlük.
    """

    # 1. Etiketleri temizle (virgülle ayrılmış string'i listeye çevir)
    candidate_labels = [label.strip() for label in labels_text.split(",")]

    # Boş etiket gelirse hata vermesin
    if not candidate_labels or candidate_labels == ['']:
        return {"Hata": 1.0}

    # 2. Model Inference (Çıkarım) Yapılıyor
    # Arka planda model şunu soruyor:
    # "Bu metin şu anlama gelir: [etiket]" hipotezi doğru mu?
    # multi_label=False -> Metin sadece BİR sınıfa ait olabilir (Softmax toplamı 1 olur).
    results = classifier(text, candidate_labels, multi_label=False)

    # 3. Sonuçları İşle (Post-processing)
    # Modelden dönen 'scores' ve 'labels' listelerini birleştiriyoruz.
    output_dict = {}
    for label, score in zip(results['labels'], results['scores']):
        output_dict[label] = score

    return output_dict

# ------------------------------------------------------------------------------
# ADIM 3: GRADIO ARAYÜZÜ (GUI)
# ------------------------------------------------------------------------------
# Mühendislik öğrencileri için önemli not:
# Gradio, ML modellerini hızlıca demo etmek için kullanılan Python tabanlı bir UI kütüphanesidir.
# Flask veya Django ile uğraşmadan web arayüzü sunar.

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🚀 Zero-Shot Türkçe Metin Sınıflandırma
        Bu model *hiçbir özel eğitim almadan*, verdiğiniz herhangi bir etiketi kullanarak metni sınıflandırabilir.
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Analiz Edilecek Metin",
                placeholder="Örn: Bu bilgisayar çok ısınıyor, fan sesi hiç durmuyor.",
                lines=3
            )
            input_labels = gr.Textbox(
                label="Kategoriler (Virgülle ayırın)",
                placeholder="Örn: Donanım Sorunu, Yazılım Hatası, Kargo Problemi",
                value="Donanım Sorunu, Yazılım Hatası, Kargo Problemi"
            )
            btn_analyze = gr.Button("Analiz Et", variant="primary")

        with gr.Column():
            # Label bileşeni, olasılıkları bar grafiği olarak gösterir
            output_chart = gr.Label(label="Tahmin Sonuçları (Güven Skoru)")

    # Butona tıklanınca ne olacağını bağlıyoruz
    btn_analyze.click(
        fn=siniflandir,              # Çalışacak Python fonksiyonu
        inputs=[input_text, input_labels], # Girdiler
        outputs=output_chart         # Çıktı nereye yazılacak
    )

    # Örnek butonları ekleyelim (Hızlı deneme için)
    gr.Examples(
        examples=[
            ["Dolar kuru bugün yine rekor kırdı, piyasalar tedirgin.", "Ekonomi, Spor, Sağlık"],
            ["Bu filmin senaryosu çok zayıf ama görsel efektler harika.", "Sinema, Tiyatro, Edebiyat"],
            ["Golü atan oyuncu ofsayt pozisyonundaydı.", "Siyaset, Futbol, Basketbol"]
        ],
        inputs=[input_text, input_labels]
    )

# Arayüzü başlat
# debug=True -> Hata alırsanız Colab konsolunda detaylı logları gösterir.
demo.launch(debug=True, share=True)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model yükleniyor... (Bu işlem modelin boyutuna göre 1-2 dakika sürebilir)


Device set to use cuda:0


Model başarıyla yüklendi! Arayüz başlatılıyor...


/tmp/ipython-input-594809040.py:80: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0118a99db961fd7f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d0118a99db961fd7f6.gradio.live


In [ ]:
# ==============================================================================
# WORKSHOP BÖLÜM 2: NLP - Müşteri Duygu Analizi (Sentiment Analysis)
# Hedef Kitle: Bilgisayar Mühendisliği Öğrencileri
# Amaç: Fine-Tuned (Özel Eğitilmiş) bir BERT modelini kullanmak ve
#       olasılık dağılımlarını (Softmax outputs) incelemek.
# ==============================================================================

# GEREKLİ KÜTÜPHANELER
# Colab'de en üstte çalıştırılacak:
# !pip install transformers gradio

import gradio as gr
from transformers import pipeline

print("Duygu Analizi Modeli Yükleniyor... (Bu işlem kısa sürer)")

# ------------------------------------------------------------------------------
# ADIM 1: FINE-TUNED MODELİ YÜKLEME
# ------------------------------------------------------------------------------
# Kullanılan Model: "savasy/bert-base-turkish-sentiment-cased"
# Bu model, özellikle Türkçe ürün ve film yorumları üzerinde eğitilmiştir.
# "Case-sensitive" (Büyük/küçük harf duyarlı) bir modeldir.
#
# top_k=None -> Tüm sınıfların (Pozitif ve Negatif) olasılıklarını döndür
# ------------------------------------------------------------------------------

analyzer = pipeline(
    task="sentiment-analysis",
    model="savasy/bert-base-turkish-sentiment-cased",
    top_k=None
)

print("Model Hazır!")

# ------------------------------------------------------------------------------
# ADIM 2: ANALİZ VE SKORLAMA FONKSİYONU
# ------------------------------------------------------------------------------
def duygu_analizi(text):
    """
    Metni alır, modele verir ve olasılıkları Türkçe etiketlerle döndürür.
    """

    # 1. Model Tahmini
    # Model çıktısı şöyledir: [[{'label': 'positive', 'score': 0.98}, ...]]
    results = analyzer(text)[0]

    # 2. Sonuçları Düzenle (Mühendisler için veri manipülasyonu)
    output_dict = {}

    # Modelin etiketlerini (positive/negative) Türkçeleştirelim
    label_map = {
        "positive": "OLUMLU 😊",
        "negative": "OLUMSUZ 😡"
    }

    for result in results:
        tr_label = label_map.get(result['label'], result['label'])
        score = result['score']
        output_dict[tr_label] = score

    return output_dict

# ------------------------------------------------------------------------------
# ADIM 3: MODERN GRADIO ARAYÜZÜ
# ------------------------------------------------------------------------------
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown(
        """
        # 🎭 Türkçe Müşteri Duygu Analizi (BERT)
        Bu model, *BERT* mimarisini kullanarak Türkçe cümlelerin duygu durumunu analiz eder.
        Sadece sonucu değil, modelin *ne kadar emin olduğunu (Confidence Score)* da görebilirsiniz.
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Müşteri Yorumu / Mesajı",
                placeholder="Örn: Ürün beklediğimden hızlı geldi ama paketleme çok kötüydü.",
                lines=4
            )

            with gr.Accordion("Mühendislik Notu: Nasıl Çalışır?", open=False):
                gr.Markdown(
                    """
                    1. Metin *Tokenization* işlemine girer (Sayısal vektörlere dönüşür).
                    2. *BERT Encoder* katmanlarından geçer (Bağlamı anlar).
                    3. Son katmanda *Softmax* fonksiyonu çalışır ve toplamı 1.0 olan olasılıklar üretilir.
                    """
                )

            btn_analyze = gr.Button("Analizi Başlat", variant="primary")

        with gr.Column():
            # Label bileşeni, dictionary döndürdüğümüzde otomatik bar grafik çizer
            output_chart = gr.Label(label="Modelin Kararı ve Olasılık Dağılımı")

    # Aksiyon
    btn_analyze.click(
        fn=duygu_analizi,
        inputs=input_text,
        outputs=output_chart
    )

    # Zorlayıcı Örnekler (Edge Cases)
    # Öğrencilerin modelin sınırlarını görmesi için "kafa karıştırıcı" örnekler seçtik.
    gr.Examples(
        examples=[
            ["Harika bir ürün, kesinlikle tavsiye ederim."],
            ["Tam bir hayal kırıklığı, param çöp oldu."],
            ["Ürün fena değil ama kargo şirketi rezaletti."], # Karışık duygu
            ["O kadar kötü ki anlatamam, bayıldım! (İroni)"], # İroni (Modeller burada zorlanır)
            ["Sanki dünyayı kurtardınız, alt tarafı bir kargo getirdiniz."] # Sarkazm
        ],
        inputs=input_text
    )

# Arayüzü başlat
demo.launch(debug=True, share=True)

Duygu Analizi Modeli Yükleniyor... (Bu işlem kısa sürer)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Device set to use cuda:0


Model Hazır!


/tmp/ipython-input-2459156114.py:66: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Glass()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f092dab32bbfd6e98e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f092dab32bbfd6e98e.gradio.live


## Çözüm:Patching
1. Vision Transformer: Patching işleminden sonrası tamamen NLP Encoder yapısı ile aynıdır.
2.  CNN(Klasik): Az veriyle çalışır.Lokale odaklanır.
3.  ViT: Resmi global görür.
4.  Multimodal:CLIP: Metin ve görüntüyü aynı anda anlayan modüller.
5.  Huggingface video:sora, edgeal:

In [ ]:
 # ==============================================================================
# WORKSHOP BÖLÜM 2: NLP - Müşteri Duygu Analizi (Sentiment Analysis)
# Hedef Kitle: Bilgisayar Mühendisliği Öğrencileri
# Amaç: Fine-Tuned (Özel Eğitilmiş) bir BERT modelini kullanmak ve
#       olasılık dağılımlarını (Softmax outputs) incelemek.
# ==============================================================================

# GEREKLİ KÜTÜPHANELER
# Colab'de en üstte çalıştırılacak:
# !pip install transformers gradio

import gradio as gr
from transformers import pipeline

print("Duygu Analizi Modeli Yükleniyor... (Bu işlem kısa sürer)")

# ------------------------------------------------------------------------------
# ADIM 1: FINE-TUNED MODELİ YÜKLEME
# ----------------------… # ==============================================================================
# WORKSHOP BÖLÜM 3: Vision - Görüntü Sınıflandırma (ViT)
# Hedef Kitle: Bilgisayar Mühendisliği Öğrencileri
# Amaç: Convolution (CNN) kullanmadan, sadece Transformer mimarisiyle
#       görüntü sınıflandırmak ve "Patching" mantığını anlamak.
# ==============================================================================

# GEREKLİ KÜTÜPHANELER
# Colab'de en üstte çalıştırılacak:
# !pip install transformers gradio torch

import gradio as gr
from transformers import pipeline
from PIL import Image

print("Vision Transformer (ViT) Modeli Yükleniyor...")

# ------------------------------------------------------------------------------
# ADIM 1: MODELİ YÜKLEME
# ------------------------------------------------------------------------------
# Kullanılan Model: "google/vit-base-patch16-224"
# - vit-base: Standart boyut (12 katmanlı Encoder, BERT-Base gibi)
# - patch16: Görüntü 16x16 piksellik karelere bölünür.
# - 224: Giriş çözünürlüğü 224x224 pikseldir.
#
# Bu model ImageNet-1k veri setinde (1000 sınıf: Kedi, Araba, Uçak vb.) eğitilmiştir.
# ------------------------------------------------------------------------------

vision_classifier = pipeline(
    task="image-classification",
    model="google/vit-base-patch16-224"
)

print("ViT Modeli Hazır!")

# ------------------------------------------------------------------------------
# ADIM 2: SINIFLANDIRMA FONKSİYONU
# ------------------------------------------------------------------------------
def goruntu_siniflandir(image):
    """
    Görüntüyü alır (PIL formatında), ViT modeline verir ve
    en yüksek olasılıklı 5 sınıfı döndürür.
    """
    if image is None:
        return None

    # 1. Pipeline Çıkarımı
    # Arka planda:
    # - Resim 224x224 boyutuna getirilir.
    # - 16x16'lık 196 parçaya bölünür (Patching).
    # - Flatten + Linear Projection ile vektörlere çevrilir.
    results = vision_classifier(image)

    # 2. Sonuçları Gradio Formatına Çevir
    # results şöyledir: [{'label': 'tabby, tabby cat', 'score': 0.9}, ...]
    output_dict = {}
    for res in results:
        label = res['label'].split(',')[0].title() # Sadece ilk ismi al (örn: "Tabby")
        score = res['score']
        output_dict[label] = score

    return output_dict

# ------------------------------------------------------------------------------
# ADIM 3: GRADIO ARAYÜZÜ
# ------------------------------------------------------------------------------
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown(
        """
        # 👁️ Vision Transformer (ViT) ile Nesne Tanıma
        Bu demo, görüntüyü pikseller bütünü olarak değil, *sıralı kelimeler (patches)* gibi işleyen ViT mimarisini kullanır.
        """
    )

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(
                type="pil",
                label="Analiz Edilecek Fotoğraf",
                height=300
            )

            # Mühendislik Öğrencileri İçin Teknik Detay Kutusu
            with gr.Accordion("Teknik Detay: CNN vs ViT", open=True):
                gr.Markdown(
                    """
                    * *Klasik Yöntem (CNN):* Piksellerin komşularıyla ilişkisine bakar (Local Features).
                    * *Bizim Yöntemimiz (ViT):* Resmi *16x16* karelere böler. Her kareyi düzleştirip (Flatten) bir vektör yapar.
                    * *Attention:* Resmin sol üst köşesindeki bir piksel ile sağ alt köşesindeki piksel arasındaki ilişkiyi tek adımda kurabilir (Global Context).
                    """
                )

            btn_run = gr.Button("Nesneyi Tanı", variant="primary")

        with gr.Column():
            output_labels = gr.Label(label="Tahminler (ImageNet-1k Sınıfları)")

    # Buton Aksiyonu
    btn_run.click(
        fn=goruntu_siniflandir,
        inputs=input_image,
        outputs=output_labels
    )

    # Hata verdiği için otomatik örnekler kaldırıldı.
    # Lütfen bilgisayarınızdan bir fotoğraf yükleyerek deneyin.

# Başlat
demo.launch(debug=True, share=True)

Duygu Analizi Modeli Yükleniyor... (Bu işlem kısa sürer)
Vision Transformer (ViT) Modeli Yükleniyor...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cuda:0
/tmp/ipython-input-4135134702.py:85: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Monochrome()) as demo:


ViT Modeli Hazır!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://85c272bb9ac927376b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
